In [40]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from shapely.geometry import Polygon, Point
import time
from datetime import datetime
from functools import reduce
import calendar
from haversine import haversine, Unit
from scipy.optimize import linear_sum_assignment
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [41]:
#Define presto credentials
presto_host = 'presto.processing.yoda.run'
presto_port = '80'
username = 'aditya.bhattar@rapido.bike'

#Create connection to presto host
connection = presto.connect(presto_host,presto_port,username = username)

In [42]:
#Load the csv with hyderabad area1 hexes
hyderabad_zone1 = pd.read_csv('/Users/rapido/Desktop/batching_demand/hyderabad/hyderabad_location_data_level1_demand_zone1.csv')
hyderabad_zone1_hexes = list(hyderabad_zone1['customer_location_hex_8'].unique())
hyderabad_zone2 = pd.read_csv('/Users/rapido/Desktop/batching_demand/hyderabad/hyderabad_location_data_level1_demand_zone2.csv')
hyderabad_zone2_hexes = list(hyderabad_zone2['customer_location_hex_8'].unique())
print(hyderabad_zone1_hexes)
print(hyderabad_zone2_hexes)

['8860a24a61fffff', '8860a24a65fffff', '8860a24a67fffff', '8860a24a6dfffff', '8860a24b51fffff', '8860a25995fffff', '8860a25997fffff', '8860a259b1fffff', '8860a259b9fffff', '8860a259bbfffff', '8860a259bdfffff', '8860a24a29fffff', '8860a24a2dfffff', '8860a24a63fffff', '8860a24a69fffff', '8860a24a6bfffff', '8860a24b53fffff', '8860a24b55fffff', '8860a24b57fffff', '8860a24b59fffff', '8860a24b5bfffff', '8860a24b5dfffff', '8860a25983fffff', '8860a25987fffff', '8860a25991fffff', '8860a25993fffff', '8860a2599dfffff', '8860a259a3fffff', '8860a259abfffff', '8860a259b3fffff', '8860a259b5fffff', '8860a259b7fffff']
['8860a25903fffff', '8860a25915fffff', '8860a2591dfffff', '8860a25939fffff', '8860a25957fffff', '8860a25901fffff', '8860a25907fffff', '8860a2590bfffff', '8860a25911fffff', '8860a25917fffff', '8860a25919fffff', '8860a25931fffff', '8860a2593bfffff', '8860a25951fffff', '8860a25953fffff', '8860a25955fffff']


In [43]:
#Combine both lists
hyderabad_zone_hexes = hyderabad_zone1_hexes + hyderabad_zone2_hexes

In [44]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, captain_location_hex_8, captain_location_latitude,
captain_location_longitude, city_name, customer_location_hex_8, drop_location_hex_8, drop_location_latitude, drop_location_longitude, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason, updated_epoch, event_type, captain_id, spd_fraud_flag
from orders.order_logs_immutable
where yyyymmdd >= '20220919'
and yyyymmdd <= '20221016'
and service_obj_service_name = 'Link'
and customer_location_hex_8 in ('8860a24a61fffff', '8860a24a65fffff', '8860a24a67fffff', '8860a24a6dfffff', '8860a24b51fffff', '8860a25995fffff',
 '8860a25997fffff', '8860a259b1fffff', '8860a259b9fffff', '8860a259bbfffff', '8860a259bdfffff', '8860a24a29fffff', '8860a24a2dfffff', '8860a24a63fffff',
  '8860a24a69fffff', '8860a24a6bfffff', '8860a24b53fffff', '8860a24b55fffff', '8860a24b57fffff', '8860a24b59fffff', '8860a24b5bfffff', '8860a24b5dfffff',
   '8860a25983fffff', '8860a25987fffff', '8860a25991fffff', '8860a25993fffff', '8860a2599dfffff', '8860a259a3fffff', '8860a259abfffff', '8860a259b3fffff',
    '8860a259b5fffff', '8860a259b7fffff', '8860a25903fffff', '8860a25915fffff', '8860a2591dfffff', '8860a25939fffff', '8860a25957fffff', '8860a25901fffff',
     '8860a25907fffff', '8860a2590bfffff', '8860a25911fffff', '8860a25917fffff', '8860a25919fffff', '8860a25931fffff', '8860a2593bfffff', '8860a25951fffff',
      '8860a25953fffff', '8860a25955fffff')
"""

#Load data into pandas table
df_copy = pd.read_sql(q, connection)

In [45]:
#Load a copy of the df
df = df_copy.copy()
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,updated_epoch,event_type,captain_id,spd_fraud_flag
0,63359c398a2e4e46739301e5,0.1890,8860a259b9fffff,17.444805,78.392357,Hyderabad,8860a24b59fffff,8860a259b9fffff,17.444805,78.392357,17.454130,78.370596,185305,18,1845,Link,18:59,4,20220929,dropped,None,1664459296746,dropped,5dc305473087fd49363caaed,False
1,63359ca03fb34379d4f5790d,1.7350,8860a24b51fffff,17.469315,78.367828,Hyderabad,8860a24a65fffff,8860a24b51fffff,17.469315,78.367828,17.450954,78.370636,185448,18,1845,Link,18:59,4,20220929,dropped,None,1664459749524,rider_rated,62d31532082b67770cb3c3c0,False
2,63359cc326e8371aef9132f3,NaN,,NaN,NaN,Hyderabad,8860a25915fffff,8860a259e9fffff,17.434517,78.428287,17.434717,78.447983,185523,18,1845,Link,18:59,4,20220929,requested,None,1664457923172,order_requested,None,None
3,633590015e5c702d2a6266e1,0.5324,8860a24a65fffff,17.452274,78.363861,Hyderabad,8860a24a65fffff,8860a259bbfffff,17.442081,78.377155,17.452225,78.364243,180057,18,1800,Link,18:00,4,20220929,arrived,None,1664454854291,arrived,6324b13b790e6a026b8657b0,None
4,63359079dc4a4a7bd2839190,NaN,8860a259bbfffff,17.441374,78.376953,Hyderabad,8860a25997fffff,8860a259bdfffff,17.453676,78.393921,17.432945,78.375443,180257,18,1800,Link,18:00,4,20220929,onTheWay,None,1664454861123,accepted,5daa6fcbe8476253eeb0156d,None


In [46]:
#Make columns for second and minute and date
df['second'] = df['hhmmss'].apply(lambda x: x[4:])
df['minute'] = df['hhmmss'].apply(lambda x: x[2:4])
df['date'] = df['yyyymmdd'].apply(lambda x: x[6:])
df['weekday_name'] = df['weekday'].apply(lambda x: calendar.day_name[(x-1)])

#Create a column to get the time of the day
df['time_period'] = np.where(df['hour'].isin(['08', '09', '10', '11']), 'morning_peak', np.where(df['hour'].isin(['17', '18', '19', '20', '21']), 'evening_peak',
    np.where(df['hour'].isin(['12, 13, 14, 15, 16']), 'afternoon', np.where(df['hour'].isin(['00', '01', '02', '03', '04', '05', '06', '07']),
     'rest_morning', 'rest_evening'))))

#Make a column to combine cust_lat_long and cap_lat_long
df['cust_lat_long'] = df[['customer_location_latitude', 'customer_location_longitude']].values.tolist()
df['cap_lat_long'] = df[['captain_location_latitude', 'captain_location_longitude']].values.tolist()

In [47]:
#Create 20-sec batches
df['second'] = df['second'].astype(float)
df['minute'] = df['minute'].astype(float)
df['20_sec_batch'] = pd.cut(df['second'], bins = [-1, 20, 40, 61], labels = ['0-20', '20-40', '40-60'])
df['count'] = 1
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,updated_epoch,event_type,captain_id,spd_fraud_flag,second,minute,date,weekday_name,time_period,cust_lat_long,cap_lat_long,20_sec_batch,count
0,63359c398a2e4e46739301e5,0.1890,8860a259b9fffff,17.444805,78.392357,Hyderabad,8860a24b59fffff,8860a259b9fffff,17.444805,78.392357,17.454130,78.370596,185305,18,1845,Link,18:59,4,20220929,dropped,None,1664459296746,dropped,5dc305473087fd49363caaed,False,5.0,53.0,29,Thursday,evening_peak,"[17.454130378589042, 78.3705958473212]","[17.444805145263672, 78.3923568725586]",0-20,1
1,63359ca03fb34379d4f5790d,1.7350,8860a24b51fffff,17.469315,78.367828,Hyderabad,8860a24a65fffff,8860a24b51fffff,17.469315,78.367828,17.450954,78.370636,185448,18,1845,Link,18:59,4,20220929,dropped,None,1664459749524,rider_rated,62d31532082b67770cb3c3c0,False,48.0,54.0,29,Thursday,evening_peak,"[17.45095443725586, 78.37063598632812]","[17.469314575195312, 78.36782836914062]",40-60,1
2,63359cc326e8371aef9132f3,NaN,,NaN,NaN,Hyderabad,8860a25915fffff,8860a259e9fffff,17.434517,78.428287,17.434717,78.447983,185523,18,1845,Link,18:59,4,20220929,requested,None,1664457923172,order_requested,None,None,23.0,55.0,29,Thursday,evening_peak,"[17.434717178344727, 78.44798278808594]","[nan, nan]",20-40,1
3,633590015e5c702d2a6266e1,0.5324,8860a24a65fffff,17.452274,78.363861,Hyderabad,8860a24a65fffff,8860a259bbfffff,17.442081,78.377155,17.452225,78.364243,180057,18,1800,Link,18:00,4,20220929,arrived,None,1664454854291,arrived,6324b13b790e6a026b8657b0,None,57.0,0.0,29,Thursday,evening_peak,"[17.452224731445312, 78.36424255371094]","[17.452274322509766, 78.36386108398438]",40-60,1
4,63359079dc4a4a7bd2839190,NaN,8860a259bbfffff,17.441374,78.376953,Hyderabad,8860a25997fffff,8860a259bdfffff,17.453676,78.393921,17.432945,78.375443,180257,18,1800,Link,18:00,4,20220929,onTheWay,None,1664454861123,accepted,5daa6fcbe8476253eeb0156d,None,57.0,2.0,29,Thursday,evening_peak,"[17.432945251464844, 78.37544250488281]","[17.4413742, 78.3769531]",40-60,1


In [48]:
#Filter the df on pinged captains
df_filter = df[(df['event_type'] == 'accepted') | (df['event_type'] == 'rider_reject') | (df['event_type'] == 'rider_busy')]

#Create a column to identify the zone
df_filter['zone_name'] = np.where(df_filter['customer_location_hex_8'].isin(hyderabad_zone1_hexes), 'zone1', 'zone2')

#Convert epoch time to timestamp
df_filter['timestamp'] = df_filter['updated_epoch'].apply(lambda x: datetime.utcfromtimestamp(x/1000))

#Sort the df by order_id and timestamp to get the first captain pinged for each order
df_filter = df_filter.sort_values(by = ['order_id', 'timestamp'], ascending=True)

#Keep only the minimum timestamp for each order it
df_min_timestamp = df_filter.groupby(['order_id'])['timestamp'].min().reset_index()
df_min_timestamp['min_time'] = 'Yes'
df_filter = df_filter.merge(df_min_timestamp, on = ['order_id', 'timestamp'], how = 'left')
df_filter['min_time'] = df_filter['min_time'].fillna('No')
df_old_FM = df_filter[df_filter['min_time'] == 'Yes']

In [50]:
#Query data from driving_distance_and_time
q_dist = """
select * from experiments.driving_distance_and_time
where city_name in ('Hyderabad')
and hex_resolution = 8
"""

#Load the data into pandas table
df_distance_copy = pd.read_sql(q_dist, connection)

In [51]:
#View the distance dataset
df_distance = df_distance_copy.copy()
df_distance.head()

,day_name,destination_hex,distance_in_km,hex_resolution,num_observations,ridetime_in_minutes,run_date,source_hex,time_period,city_name
0,Wednesday,8860a24b5bfffff,5.36600,8,15,12.541867,20221101,8860a24b49fffff,rest_morning,Hyderabad
1,Saturday,8860a2599dfffff,2.64450,8,43,3.742292,20221101,8860a2599dfffff,rest_evening,Hyderabad
2,Monday,8860a25a5dfffff,4.58650,8,23,8.533942,20221101,8860a25a0dfffff,morning_peak,Hyderabad
3,Saturday,8860a25845fffff,1.96925,8,236,5.855867,20221101,8860a25b13fffff,evening_peak,Hyderabad
4,Monday,8860a25a27fffff,6.56975,8,122,17.240504,20221101,8860a25b37fffff,afternoon,Hyderabad


In [52]:
#Filter df_distance only on columns and hexes required
df_distance_req = df_distance[['day_name', 'destination_hex', 'distance_in_km', 'source_hex', 'time_period']]

df_distance_req = df_distance_req[df_distance_req['destination_hex'].isin(hyderabad_zone_hexes)]
df_distance_req['zone_name'] = np.where(df_distance_req['destination_hex'].isin(hyderabad_zone1_hexes), 'zone1', 'zone2')
df_distance_req_zone1 = df_distance_req[df_distance_req['zone_name'] == 'zone1']
df_distance_req_zone2 = df_distance_req[df_distance_req['zone_name'] == 'zone2']

In [53]:
#Create a column to combine customer and captain hex
df_old_FM['captain_customer_hex'] = df_old_FM['captain_location_hex_8'] + df_old_FM['customer_location_hex_8']

#Rename columns in df_distance to match the column name in df_old_FM
df_distance = df_distance.rename(columns={'day_name':'weekday_name'})
df_distance['captain_customer_hex'] = df_distance['source_hex'] + df_distance['destination_hex']

#Create a time_period column in the df
df_old_FM['time_period'] = np.where(df_old_FM['hour'].isin(['08', '09', '10', '11']), 'morning_peak',
 np.where(df_old_FM['hour'].isin(['17', '18', '19', '20', '21']), 'evening_peak',
 np.where(df_old_FM['hour'].isin(['12, 13, 14, 15, 16']), 'afternoon',
 np.where(df_old_FM['hour'].isin(['00', '01', '02', '03', '04', '05', '06', '07']), 'rest_morning', 'rest_evening'))))

#Merge the dfs to get the distance
df_old_FM = df_old_FM.merge(df_distance[['captain_customer_hex', 'time_period', 'weekday_name', 'distance_in_km']],
 on = ['captain_customer_hex', 'time_period', 'weekday_name'], how = 'left')
df_old_FM['distance_in_km'] = df_old_FM['distance_in_km'].fillna('NA')
df_old_FM.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,updated_epoch,event_type,captain_id,spd_fraud_flag,second,minute,date,weekday_name,time_period,cust_lat_long,cap_lat_long,20_sec_batch,count,zone_name,timestamp,min_time,captain_customer_hex,distance_in_km
0,632763a87ff9294e2753d015,NaN,8860a24a67fffff,17.449480,78.355345,Hyderabad,8860a24a67fffff,8860a24a65fffff,17.450116,78.370557,17.448740,78.355888,000000,00,0000,Link,00:00,1,20220919,onTheWay,None,1663525809609,accepted,63204eefdc7efadb6517361f,None,0.0,0.0,19,Monday,rest_morning,"[17.448740005493164, 78.35588836669922]","[17.4494801, 78.3553445]",0-20,1,zone1,2022-09-18 18:30:09.609,Yes,8860a24a67fffff8860a24a67fffff,0.354
1,632763af853c020ac793ba6a,NaN,8860a259b9fffff,17.442532,78.391635,Hyderabad,8860a259b9fffff,8860a259e9fffff,17.435646,78.427657,17.441948,78.391975,000007,00,0000,Link,00:00,1,20220919,onTheWay,None,1663525812025,accepted,5dbcafdd3405484920146a2f,None,7.0,0.0,19,Monday,rest_morning,"[17.44194793701172, 78.39197540283203]","[17.4425319, 78.3916347]",0-20,1,zone1,2022-09-18 18:30:12.025,Yes,8860a259b9fffff8860a259b9fffff,1.422
2,632763b77ff9294e2753d028,NaN,8860a2593bfffff,17.444573,78.444315,Hyderabad,8860a25915fffff,8860a2593dfffff,17.450403,78.452104,17.438328,78.448250,000015,00,0000,Link,00:00,1,20220919,onTheWay,None,1663525833692,accepted,5d1c7f153b752c45cf94d4b7,None,15.0,0.0,19,Monday,rest_morning,"[17.43832778930664, 78.44824981689453]","[17.4445731, 78.4443151]",0-20,1,zone2,2022-09-18 18:30:33.692,Yes,8860a2593bfffff8860a25915fffff,2.08
3,632763be745615349b014149,NaN,8860a259b1fffff,17.452413,78.380229,Hyderabad,8860a259b1fffff,8860a24865fffff,17.500359,78.338701,17.452072,78.380486,000022,00,0000,Link,00:00,1,20220919,onTheWay,None,1663525838651,accepted,5d503f2d55fbf50d45ef8eab,None,22.0,0.0,19,Monday,rest_morning,"[17.452072143554688, 78.38048553466797]","[17.4524128, 78.380229]",20-40,1,zone1,2022-09-18 18:30:38.651,Yes,8860a259b1fffff8860a259b1fffff,1.6455
4,632763c5745615349b01414e,NaN,8860a259b9fffff,17.444310,78.390796,Hyderabad,8860a259b9fffff,8860a259b9fffff,17.443272,78.390510,17.443068,78.390427,000029,00,0000,Link,00:00,1,20220919,new,None,1663525859997,rider_busy,632282ce0c0101761a087559,None,29.0,0.0,19,Monday,rest_morning,"[17.44306755065918, 78.39042663574219]","[17.444309681614705, 78.39079588651657]",20-40,1,zone1,2022-09-18 18:30:59.997,Yes,8860a259b9fffff8860a259b9fffff,1.422


In [54]:
#Calculate the haversine distance
cap_lat_long = df_old_FM['cap_lat_long'].values.tolist()
cust_lat_long = df_old_FM['cust_lat_long'].values.tolist()

distances = []
for x in range(0, len(cap_lat_long)):
    individual_distance = round(haversine(cap_lat_long[x], cust_lat_long[x]), 3)
    distances.append(individual_distance)

df_old_FM['haversine_distance'] = distances
df_old_FM['old_FM'] = np.where(df_old_FM['distance_in_km'] == 'NA', df_old_FM['haversine_distance'], df_old_FM['distance_in_km'])

#Get the average FM by date and hour
df_old_FM_average = df_old_FM.groupby(['zone_name', 'yyyymmdd', 'hour'])['old_FM'].median().reset_index()
df_old_FM_average.head(10)

,zone_name,yyyymmdd,hour,old_FM
0,zone1,20220919,00,1.523500
1,zone1,20220919,01,1.645500
2,zone1,20220919,02,1.645500
3,zone1,20220919,03,1.718000
4,zone1,20220919,04,1.754500
5,zone1,20220919,05,1.796875
6,zone1,20220919,06,1.645500
7,zone1,20220919,07,1.645500
8,zone1,20220919,08,1.911725
9,zone1,20220919,09,2.175500


In [55]:
#Calculate the FM as per batching

#Create seperate dfs for zone1 and zone2
df_filter_zone1 = df_filter[df_filter['zone_name'] == 'zone1']
df_filter_zone2 = df_filter[df_filter['zone_name'] == 'zone2']

In [56]:
#Filter the df by hour to get hourly reduction in fm
hours_zone1 = {}
dates_zone1 = {}
for date in df_filter_zone1['date'].unique().tolist():
    df_date = df_filter_zone1[df_filter_zone1['date'] == date]
    for hour in df_date['hour'].unique().tolist():
        df_hourly = df_date[df_date['hour'] == hour]
        df_distance_day = df_distance_req_zone1[df_distance_req_zone1['day_name'] == df_hourly['weekday_name'].values[0]]

        #Get the captains available for every 20_sec_batch
        captains_available = df_hourly.groupby(['hour', 'minute', '20_sec_batch'])['captain_location_hex_8'].apply(list).reset_index()
        captains_available2 = df_hourly.groupby(['hour', 'minute', '20_sec_batch'])['cap_lat_long'].apply(list).reset_index()
        captains_available = captains_available.rename(columns = {'captain_location_hex_8':'cap_location_hexes'})
        captains_available2 = captains_available2.rename(columns = {'cap_lat_long':'cap_location_lat_long'})
        #captains_available['number_of_captains'] = captains_available['cap_location_hexes'].apply(lambda x: len(x))
        captains_available = captains_available.merge(captains_available2, on = ['hour', 'minute', '20_sec_batch'], how = 'left')

        #Merge the captain locations with our df
        df_hourly = df_hourly.merge(captains_available, on = ['hour', 'minute', '20_sec_batch'], how = 'left')
        
        #Filter the distance df based on the hour
        if hour in ['08', '09', '10', '11']:
            df_distance_filter = df_distance_day[df_distance_day['time_period'] == 'morning_peak']
        elif hour in ['17', '18', '19', '20', '21']:
            df_distance_filter = df_distance_day[df_distance_day['time_period'] == 'evening_peak']
        elif hour in ['12, 13, 14, 15, 16']:
            df_distance_filter = df_distance_day[df_distance_day['time_period'] == 'afternoon']
        elif hour in ['00', '01', '02', '03', '04', '05', '06', '07']:
            df_distance_filter = df_distance_day[df_distance_day['time_period'] == 'rest_morning']
        else:
            df_distance_filter = df_distance_day[df_distance_day['time_period'] == 'rest_evening']

        #Combine the source and destination hex and make a dictionary to get distance between each hex combo
        df_distance_filter['source_destination'] = df_distance_filter['source_hex'] + df_distance_filter['destination_hex']
        distance_dict = dict(zip(df_distance_filter['source_destination'], df_distance_filter['distance_in_km']))

        #Get the distance between customer and each captain available
        customer_hexes = df_hourly['customer_location_hex_8'].values.tolist()
        captain_hexes = df_hourly['cap_location_hexes'].values.tolist()
        cap_lat_long = df_hourly['cap_location_lat_long'].values.tolist()
        cust_lat_long = df_hourly['cust_lat_long'].values.tolist()

        i = 0
        captain_distances = []
        for x in captain_hexes:
            customer_distance = []
            for hex in range(0, len(x)):
                key_hex = x[hex] + customer_hexes[i]
                if key_hex in distance_dict:
                    individual_distance = distance_dict[key_hex]
                else:
                    individual_distance = round(haversine(cap_lat_long[i][hex], cust_lat_long[i]), 3)
                customer_distance.append(individual_distance)
            captain_distances.append(customer_distance)
            i += 1

        #Add columns for captain distances and etas to each customer
        df_hourly['captain_distances'] = captain_distances

        #Get the nxn matrix for each 20_sec_batch
        df_batches = df_hourly.groupby(['minute', '20_sec_batch'])['captain_distances'].apply(list).reset_index()
        df_batches = df_batches.dropna()

        #Get the row_index and column index suitable for each customer
        optimal_distance = []
        for x in df_batches['captain_distances'].values.tolist():
            try:
                row_ind, col_ind = linear_sum_assignment(x)
            except Exception as e:
                continue
            optimal_distance.append(col_ind)

        #Get the optimal distance for each customer
        captain_customer_dist = df_batches['captain_distances'].values.tolist()
        i = 0
        min_distances = []
        for x in captain_customer_dist:
            j = 0
            for y in x:
                if type(y) == float:
                    continue
                min_distance = y[optimal_distance[i][j]]
                j += 1
                min_distances.append(min_distance)
            i += 1

        #Add the new_dist to the df
        df_hourly['new_FM'] = min_distances

        #Get the mean FM for each 20_sec_batch (old and new)
        fm_min = df_hourly.groupby(['minute', '20_sec_batch'])['new_FM'].median().reset_index()
        fm_min_value = fm_min['new_FM'].median()
        hours_zone1[hour] = fm_min_value
    dates_zone1[date] = hours_zone1
    print('done for date {}.'.format(date))

done for date 19.
done for date 20.
done for date 21.
done for date 22.
done for date 23.
done for date 24.
done for date 25.
done for date 26.
done for date 27.
done for date 28.
done for date 29.
done for date 30.
done for date 01.
done for date 02.
done for date 03.
done for date 04.
done for date 05.
done for date 06.
done for date 07.
done for date 08.
done for date 09.
done for date 10.
done for date 11.
done for date 12.
done for date 13.
done for date 14.
done for date 15.
done for date 16.


In [57]:
#Filter the df by hour to get hourly reduction in fm
hours_zone2 = {}
dates_zone2 = {}
for date in df_filter_zone2['date'].unique().tolist():
    df_date = df_filter_zone2[df_filter_zone2['date'] == date]
    for hour in df_date['hour'].unique().tolist():
        df_hourly = df_date[df_date['hour'] == hour]
        df_distance_day = df_distance_req_zone2[df_distance_req_zone2['day_name'] == df_hourly['weekday_name'].values[0]]

        #Get the captains available for every 20_sec_batch
        captains_available = df_hourly.groupby(['hour', 'minute', '20_sec_batch'])['captain_location_hex_8'].apply(list).reset_index()
        captains_available2 = df_hourly.groupby(['hour', 'minute', '20_sec_batch'])['cap_lat_long'].apply(list).reset_index()
        captains_available = captains_available.rename(columns = {'captain_location_hex_8':'cap_location_hexes'})
        captains_available2 = captains_available2.rename(columns = {'cap_lat_long':'cap_location_lat_long'})
        #captains_available['number_of_captains'] = captains_available['cap_location_hexes'].apply(lambda x: len(x))
        captains_available = captains_available.merge(captains_available2, on = ['hour', 'minute', '20_sec_batch'], how = 'left')

        #Merge the captain locations with our df
        df_hourly = df_hourly.merge(captains_available, on = ['hour', 'minute', '20_sec_batch'], how = 'left')
        
        #Filter the distance df based on the hour
        if hour in ['08', '09', '10', '11']:
            df_distance_filter = df_distance_day[df_distance_day['time_period'] == 'morning_peak']
        elif hour in ['17', '18', '19', '20', '21']:
            df_distance_filter = df_distance_day[df_distance_day['time_period'] == 'evening_peak']
        elif hour in ['12, 13, 14, 15, 16']:
            df_distance_filter = df_distance_day[df_distance_day['time_period'] == 'afternoon']
        elif hour in ['00', '01', '02', '03', '04', '05', '06', '07']:
            df_distance_filter = df_distance_day[df_distance_day['time_period'] == 'rest_morning']
        else:
            df_distance_filter = df_distance_day[df_distance_day['time_period'] == 'rest_evening']

        #Combine the source and destination hex and make a dictionary to get distance between each hex combo
        df_distance_filter['source_destination'] = df_distance_filter['source_hex'] + df_distance_filter['destination_hex']
        distance_dict = dict(zip(df_distance_filter['source_destination'], df_distance_filter['distance_in_km']))

        #Get the distance between customer and each captain available
        customer_hexes = df_hourly['customer_location_hex_8'].values.tolist()
        captain_hexes = df_hourly['cap_location_hexes'].values.tolist()
        cap_lat_long = df_hourly['cap_location_lat_long'].values.tolist()
        cust_lat_long = df_hourly['cust_lat_long'].values.tolist()

        i = 0
        captain_distances = []
        for x in captain_hexes:
            customer_distance = []
            for hex in range(0, len(x)):
                key_hex = x[hex] + customer_hexes[i]
                if key_hex in distance_dict:
                    individual_distance = distance_dict[key_hex]
                else:
                    individual_distance = round(haversine(cap_lat_long[i][hex], cust_lat_long[i]), 3)
                customer_distance.append(individual_distance)
            captain_distances.append(customer_distance)
            i += 1

        #Add columns for captain distances and etas to each customer
        df_hourly['captain_distances'] = captain_distances

        #Get the nxn matrix for each 20_sec_batch
        df_batches = df_hourly.groupby(['minute', '20_sec_batch'])['captain_distances'].apply(list).reset_index()
        df_batches = df_batches.dropna()

        #Get the row_index and column index suitable for each customer
        optimal_distance = []
        for x in df_batches['captain_distances'].values.tolist():
            try:
                row_ind, col_ind = linear_sum_assignment(x)
            except Exception as e:
                continue
            optimal_distance.append(col_ind)

        #Get the optimal distance for each customer
        captain_customer_dist = df_batches['captain_distances'].values.tolist()
        i = 0
        min_distances = []
        for x in captain_customer_dist:
            j = 0
            for y in x:
                if type(y) == float:
                    continue
                min_distance = y[optimal_distance[i][j]]
                j += 1
                min_distances.append(min_distance)
            i += 1

        #Add the new_dist to the df
        df_hourly['new_FM'] = min_distances

        #Get the mean FM for each 20_sec_batch (old and new)
        fm_min = df_hourly.groupby(['minute', '20_sec_batch'])['new_FM'].median().reset_index()
        fm_min_value = fm_min['new_FM'].median()
        hours_zone2[hour] = fm_min_value
    dates_zone2[date] = hours_zone2
    print('done for date {}.'.format(date))

done for date 19.
done for date 20.
done for date 21.
done for date 22.
done for date 23.
done for date 24.
done for date 25.
done for date 26.
done for date 27.
done for date 28.
done for date 29.
done for date 30.
done for date 01.
done for date 02.
done for date 03.
done for date 04.
done for date 05.
done for date 06.
done for date 07.
done for date 08.
done for date 09.
done for date 10.
done for date 11.
done for date 12.
done for date 13.
done for date 14.
done for date 15.
done for date 16.


In [58]:
#Create a df from the dates dict
new_FM_zone1 = pd.DataFrame(dates_zone1).T
new_FM_zone1 = new_FM_zone1.reset_index()
new_FM_zone1 = pd.melt(new_FM_zone1, id_vars='index')
new_FM_zone1.columns = ['date', 'hour', 'new_FM']
new_FM_zone1 = new_FM_zone1.sort_values(by = ['date', 'hour'], ascending=True)
new_FM_zone1['zone_name'] = 'zone1'

new_FM_zone2 = pd.DataFrame(dates_zone2).T
new_FM_zone2 = new_FM_zone2.reset_index()
new_FM_zone2 = pd.melt(new_FM_zone2, id_vars='index')
new_FM_zone2.columns = ['date', 'hour', 'new_FM']
new_FM_zone2 = new_FM_zone2.sort_values(by = ['date', 'hour'], ascending=True)
new_FM_zone2['zone_name'] = 'zone2'

df_new_FM = pd.concat([new_FM_zone1, new_FM_zone2], axis=0)

#Merge old and new_FM
df_old_FM_average['date'] = df_old_FM_average['yyyymmdd'].apply(lambda x: x[6:])
hourly_decrease = df_old_FM_average.merge(df_new_FM, on = ['zone_name', 'date', 'hour'], how = 'left')
hourly_decrease['change_in_FM_%'] = round((hourly_decrease['new_FM']/hourly_decrease['old_FM'] - 1), 4)
hourly_decrease

,zone_name,yyyymmdd,hour,old_FM,date,new_FM,change_in_FM_%
0,zone1,20220919,00,1.52350,19,1.434669,-0.0583
1,zone1,20220919,01,1.64550,19,1.730000,0.0514
2,zone1,20220919,02,1.64550,19,1.976544,0.2012
3,zone1,20220919,03,1.71800,19,1.941250,0.1299
4,zone1,20220919,04,1.75450,19,1.997875,0.1387
...,...,...,...,...,...,...,...
1339,zone2,20221016,19,1.66475,16,1.779600,0.0690
1340,zone2,20221016,20,1.65375,16,1.734563,0.0489
1341,zone2,20221016,21,1.66050,16,1.752525,0.0554
1342,zone2,20221016,22,1.73400,16,1.814500,0.0464


In [59]:
hourly_decrease.groupby(['zone_name'])['change_in_FM_%'].median()

zone_name
zone1    0.07380
zone2    0.06425
Name: change_in_FM_%, dtype: float64

In [60]:
hourly_decrease.to_csv('change_in_FM_Batching.csv', index = False)